<a href="https://colab.research.google.com/github/lakatosgabor/lama_qa_hun/blob/main/lama_QA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install

In [ ]:
!sudo apt install pciutils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpci3 pci.ids
The following NEW packages will be installed:
  libpci3 pci.ids pciutils
0 upgraded, 3 newly installed, 0 to remove and 35 not upgraded.
Need to get 343 kB of archives.
After this operation, 1,581 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 pci.ids all 0.0~2022.01.22-1ubuntu0.1 [251 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libpci3 amd64 1:3.7.0-6 [28.9 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 pciutils amd64 1:3.7.0-6 [63.6 kB]
Fetched 343 kB in 1s (655 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 3.)
debconf: falling back to frontend: Readline
debconf: un

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> NVIDIA GPU installed.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
!pip install -qq pyngrok ollama

In [ ]:
!nvidia-smi
import os
os.environ["OLLAMA_NUM_GPU_LAYERS"] = "100"

Tue Aug 12 16:31:15 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   45C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

### Import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import IPython
import subprocess
from pyngrok import ngrok
from google.colab import userdata
from typing import Any, Dict, List, Optional
import requests
import json

### Lama & Ngrok szerver

In [ ]:
def start_ollama_server() -> None:
    """Starts the Ollama server."""
    subprocess.Popen(['ollama', 'serve'])
    print("Ollama server started.")


def check_ollama_port(port: str) -> None:
    """Check if Ollama server is running at the specified port."""
    try:
        subprocess.run(['sudo', 'lsof', '-i', '-P', '-n'], check=True, capture_output=True, text=True)
        if any(f":{port} (LISTEN)" in line for line in subprocess.run(['sudo', 'lsof', '-i', '-P', '-n'], capture_output=True, text=True).stdout.splitlines()):
            print(f"Ollama is listening on port {port}")
        else:
            print(f"Ollama does not appear to be listening on port {port}.")
    except subprocess.CalledProcessError as e:
         print(f"Error checking Ollama port: {e}")


def setup_ngrok_tunnel(port: str) -> ngrok.NgrokTunnel:
    """Sets up an ngrok tunnel.

    Args:
        port: The port to tunnel.

    Returns:
        The ngrok tunnel object.

    Raises:
        RuntimeError: If the ngrok authtoken is not set.
    """
    ngrok_auth_token = "30PNgTX5ijqAbIicSn0ZHwEOOLU_3hQRpPf6wKHW25mTj462b"
    if not ngrok_auth_token:
        raise RuntimeError("NGROK_AUTHTOKEN is not set.")

    ngrok.set_auth_token(ngrok_auth_token)
    tunnel = ngrok.connect(port, host_header=f'localhost:{port}')
    print(f"ngrok tunnel created: {tunnel.public_url}")
    return tunnel

In [ ]:
NGROK_PORT = '11434'
ngrok_tunnel = setup_ngrok_tunnel(NGROK_PORT)

ngrok tunnel created: https://aadd4cda17fe.ngrok-free.app


In [ ]:
start_ollama_server()

check_ollama_port(NGROK_PORT)

Ollama server started.
Ollama is listening on port 11434


### HUN Modell betöltés

In [ ]:
!ollama pull jobautomation/OpenEuroLLM-Hungarian
#!ollama pull llama3

### Q&A

In [ ]:
url = "https://029d4a167895.ngrok-free.app/api/generate"

payload = {
    "model": "llama3",
    "prompt": "Magyarország fővárosa?"
}

response = requests.post(url, json=payload, stream=True)
print(response)

# Válasz darabok összefűzése
full_response = ""

for line in response.iter_lines():
    if line:
        data = json.loads(line.decode("utf-8"))
        full_response += data.get("response", "")
        if data.get("done", False):
            break

print(full_response)


### RAG

In [ ]:
!pip install --upgrade langchain langchain-community langchain-core langchain-ollama chromadb

In [ ]:
import json
import requests
from typing import Optional, List, Any
from langchain_core.callbacks.manager import CallbackManagerForLLMRun
from langchain_core.language_models import LLM
from langchain_core.outputs import Generation
from langchain_community.vectorstores import Chroma
from langchain_ollama import OllamaEmbeddings
from langchain.chains import RetrievalQA
from langchain.docstore.document import Document
import pandas as pd
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate



class CustomLLM(LLM):
    model: str
    url: str

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
    ) -> str:
        payload = {
            "model": self.model,
            "prompt": prompt,
        }

        response = requests.post(self.url, json=payload, stream=True)
        print(response)

        full_response = ""
        for line in response.iter_lines():
            if line:
                data = json.loads(line.decode("utf-8"))
                full_response += data.get("response", "")
                if data.get("done", False):
                    break

        return full_response

    @property
    def _llm_type(self) -> str:
        return "custom_ngrok_llm"

# 1. CSV beolvasása
df = pd.read_csv("/content/drive/MyDrive/PhD/Saját cikkek/lama/adatbazis.csv", sep=";")


# 2. Dokumentumok létrehozása
documents = [
    Document(page_content=f"Kérdés: {row['kerdes']}\nVálasz: {row['valasz']}")
    for _, row in df.iterrows()
]

# 3. Embedding és Chroma adatbázis létrehozása
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
db = Chroma.from_documents(documents, embeddings)

# 4. Egyedi LLM példány létrehozása az Ngrok API-hoz
llm = CustomLLM(
    model="jobautomation/openeurollm-hungarian", #model="llama3",
    url="https://aadd4cda17fe.ngrok-free.app/api/generate"
)

# Prompt sablon létrehozása
prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template="""
          Mindig magyar nyelven válaszolj!
          A következő információkat kaptad az adatbázisból:
          {context}

          Ha a kérdésre választ találsz az információkban akkor generálj részletes, kiegészített választ.
          Ha NINCS benne információ, válaszolj pontosan ezzel: "Erről nincs információm az adatbázisban."

          Kérdés: {question}
          Válasz:
        """
)


'''prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template="""
              Mindig magyar nyelven válaszolj, függetlenül attól, hogy a kérdés milyen nyelvű!
              Használd a következő kontextust a válaszhoz:
              {context}

              Kérdés: {question}
              Válasz:
            """
)
'''

qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=db.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt_template}
)

def kerdezz_valaszolj(query: str, threshold: float = 0.6):
    # 1. Keresés hasonlóság alapján
    retriever_results = db.similarity_search_with_score(query, k=1)  # csak a legjobb találat

    if not retriever_results or retriever_results[0][1] < threshold:
        return "Erről nincs információm az adatbázisban."

    # 2. Ha van releváns találat, átadjuk a QA láncnak
    result = qa.invoke(query)
    return result["result"]

# 7. Példakérdés
query = "Ki a rendszergazda, és milyen jelszót kell beállítani az informatikai rendszerekhez?"
valasz = kerdezz_valaszolj(query)
print(valasz)